Proyecto Individual 1: ML Ops/ETL

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import ast
import re


In [58]:
#Importo el dataset Movies
df=pd.read_csv('/Users/florencialuppi/Desktop/DS/P1/Dataset/movies_dataset.csv')

/var/folders/zr/s7c6bb410zbdsw3jm8bptp5m0000gn/T/ipykernel_14000/1489510585.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/Users/florencialuppi/Desktop/DS/P1/Dataset/movies_dataset.csv')


In [59]:
#Importo el dataset Credits
df1=pd.read_csv('/Users/florencialuppi/Desktop/DS/P1/Dataset/credits.csv')

In [60]:
df.tail(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [61]:
#Desanidar belongs_to_collection (Definitivo)

#extraemos el nombre de la columna
patron = r"'name': '([^']*)"
#extreemos nom coleccion
df['collection_name'] = df['belongs_to_collection'].str.extract(patron, flags= re.IGNORECASE )
#columna valores individuales
df['collection_unique']=df['collection_name'].str.split(', ')
#se expanden los registros
df = df.explode('collection_unique').reset_index(drop= True)
#se elimina la columna 'coleccion_name
df.drop('collection_name', axis=1, inplace= True)

In [62]:
#Desanidar genres (Definitivo)

#convertimos en lista los valores de la columna 'genres'
df['genres']=df['genres'].apply(ast.literal_eval)

# aquí vamos a extaer solamente los nombres de los géneros con la funcin 'df extract_genres_name()'
def extract_genres_name(genres_list):
    genre_name = [genre['name'] for genre in genres_list]
    return ', '.join(genre_name)
#ahora vamos aplicar la función a la columna
df['genres'] = df['genres'].apply(extract_genres_name)

# separamos los valores unicos de la columna 'genres'
df['genre_unique'] = df['genres'].str.split(', ')

#explode
df = df.explode('genre_unique').reset_index(drop = True)

#vamos a elimina la columna 'genres'
df.drop('genres', axis=1, inplace=True)

In [63]:
#Desanidar production_countries (Definitivo)

# Convertir la columna 'production_countries' en una lista de diccionarios
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

# Crear la columna 'countries_name' a partir de la lista de diccionarios en 'production_countries'
df['prod_countries'] = df['production_countries'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else None)

# Eliminar la columna 'production_countries' 
df = df.drop(columns=['production_countries'])


In [64]:
# Desanidar production_companies (Definitivo)

# Convertir la columna 'production_companies' en una lista de diccionarios
df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

# Crear las columnas 'prod_companies_id' y 'prod_companies_name' a partir de la lista de diccionarios en 'production_companies'
df['prod_companies_id'] = df['production_companies'].apply(lambda x: x[0].get('id') if isinstance(x, list) and len(x) > 0 else None)
df['prod_companies_name'] = df['production_companies'].apply(lambda x: x[0].get('name') if isinstance(x, list) and len(x) > 0 else None)

# Eliminar la columna 'production_companies' 
df = df.drop(columns=['production_companies'])


In [65]:
#Rellenar con 0, los  valores nulos de los campos revenue y budget

df['revenue'] = df['revenue'].fillna(0)

In [66]:
#Los valores nulos del campo release date deben eliminarse.

df['release_date'] = df['release_date'].dropna()

In [67]:
#De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [68]:
#Se crea la columna 'release_year'
df['release_year'] = df['release_date'].dt.year

In [69]:
# Convertir la columna 'columna_objeto' a tipo float con manejo de errores y valores no numéricos
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Verificar si hay valores no numéricos (NaN) después de la conversión
valores_no_numericos = df[df['budget'].isna()]['budget']

#si hay valores no numéricos, reemplazarlos por 0
df['budget'] = df['budget'].fillna(0)


In [70]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos 
#disponibles para calcularlo, deberá tomar el valor 0.
df['return'] = df['revenue'] / df['budget']


In [71]:
#Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [72]:
#Arreglo de valores en columna 'id'
problematic_value = df.loc[df['id'].apply(lambda x: not x.isdigit()), 'id'].unique()
print(problematic_value)


['1997-08-20' '2012-09-29' '2014-01-01']


In [73]:
#Elimino valores problematicos
problematic_ids = ['1997-08-20', '2012-09-29', '2014-01-01']
df = df[~df['id'].isin(problematic_ids)]


In [74]:
#Cambiar datatype de id de str a int
df['id'] = df['id'].astype('int64')

In [75]:
#Cambiar datatype de title a str
df['title'] = df['title'].astype('str')

In [76]:
#Extraigo Director de crew

# Convertir la columna 'crew' en una lista de diccionarios
df1['crew'] = df1['crew'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

# Extraer 'Director' en una nueva columna
df1['director'] = df1['crew'].apply(lambda x: [d['name'] for d in x if d['job'] == 'Director'][0] if isinstance(x, list) and any(d['job'] == 'Director' for d in x) else None)

# Eliminar la columna 'crew' 
df1 = df1.drop(columns=['crew'])



In [77]:
#Concateno los dos dataframes (para tener Director en el df original)
df_merged = pd.concat([df, df1], axis=1, join='inner')

In [78]:
#Elimino columna belongs_to_collection y cast
df_merged = df_merged.drop(columns=['belongs_to_collection', 'cast'])

In [82]:
#Reemplazar nan de title ''
df_merged['title'].fillna('', inplace=True)

In [83]:
df_merged.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'collection_unique',
       'genre_unique', 'prod_countries', 'prod_companies_id',
       'prod_companies_name', 'release_year', 'return', 'id', 'director'],
      dtype='object')

In [85]:
#Guardo el df en un csv ya transformado
df_merged.to_csv('/Users/florencialuppi/Desktop/DS/P1/Dataset/movies_dataset_transformed.csv', index=False)


In [86]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45473 entries, 0 to 45475
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   budget               45473 non-null  float64       
 1   id                   45473 non-null  int64         
 2   original_language    45472 non-null  object        
 3   overview             45284 non-null  object        
 4   popularity           45470 non-null  object        
 5   release_date         45452 non-null  datetime64[ns]
 6   revenue              45473 non-null  float64       
 7   runtime              45438 non-null  float64       
 8   spoken_languages     45470 non-null  object        
 9   status               45443 non-null  object        
 10  tagline              28679 non-null  object        
 11  title                45473 non-null  object        
 12  vote_average         45470 non-null  float64       
 13  vote_count           45470 non-null 

In [87]:
#nan en director
df_merged['director'].isna().sum()


887